In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2023-08-15 21:45:32.157214: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 21:45:33.029352: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1


2023-08-15 21:45:34.094921: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-15 21:45:34.230293: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-15 21:45:34.230344: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [ ]:
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces

class StockTradingEnvironment(gym.Env):
    def __init__(self, df):
        super(StockTradingEnvironment, self).__init__()

        self.df = df
        self.current_step = 0
        self.current_capital = 10000
        self.stock_owned = 0

        self.action_space = spaces.Discrete(3)  # BUY, SELL, NO ACTION
        self.observation_space = spaces.Box(low=0, high=1, shape=(len(self.df.columns),), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        self.current_capital = 10000
        self.stock_owned = 0
        return self._next_observation()

    def _next_observation(self):
        return self.df.iloc[self.current_step].values

    def step(self, action):
        prev_capital = self.current_capital

        if action == 0:  # BUY
            self.stock_owned += 1
            self.current_capital -= self.df.iloc[self.current_step]["Close"]
        elif action == 1:  # SELL
            self.stock_owned -= 1
            self.current_capital += self.df.iloc[self.current_step]["Close"]

        self.current_step += 1

        done = self.current_step == len(self.df) - 1
        reward = self.current_capital - prev_capital - (0.001 * abs(action-1))  # discourage frequent trading by a small penalty

        return self._next_observation(), reward, done, {}

    def render(self, mode='human', close=False):
        pass
